## Introduction
In this example, we show that how gear can be applied to find the bounds of error of a digital signal processing circuit throught the fixed-number lengths of the internal calculation.

The errors of computation of a digital signal processing circuit come from three sources: truncation error, quantization error, and inaccurate source.
1.  Truncation Error:
The truncation error is the error when a fixed-point number is converted from a longer representation to a shorter representation, which means the bits are not sufficient to represent the number, and thus the number must be truncated.
The truncation error occur when the result of an operation is connect to a signal with different word length.
2. Quantization Error:
The quantization error occurs when the an analog signal is converted into a digital signal, or an algorithm requires real numbers as its coefficients. As the fixed number cannot represent any real number with infinite precision. The number has to be quantized and thus are not as the exact value as the analog signal.

3. Inaccurate Source:
This error comes from the measurement in the analog signal because of noise and non-ideal effect on the sensor, and could be seen as a random noise on the analog signal.

The determination of the number of bits for fixed-point number requires tradeoff between the accuracy and the performance of the circuit.
Power, timing, and area of the circuit can benefit from reducing the number of bits for representation of fixed-point number as the gate count can be reduced, which results in shorter critical path length for optimization and smaller core footprint.
However, the accuracy of the signal processing decreases as more error is introduced in the computation and propagated to the output of the circuit.

Therefore, in digital signal processing circuit design. Exploration of the word lengths for all fixed-point number is a crucial stage from algorithm design to circuit implementation for optimizing the circuit performance while satisfying the system requirements.


## Preliminaries
#### Representation of Fixed-point Number

A word length of a fixed-point number can be described as a tuple $(n, p)$, where $n$ denotes the number of bits, and $p$ is the position of the binary point, with 0 position means the position before the most significant bit.

For example, A fixed-point number with word length $(5,2)$ with bit value "11011" means the binary number $11.011$, which is equivalent to $3.375$ in decimal.

First, we import some data structure for holding the wordlength, and the apis of gear.

In [1]:
from gear.terms.polyhedra.loaders import readContract, writeContract
from tool import PortWordLength

In [2]:
# Definition of value_num method in PortWordLength class
#    def value_num(self):
#        return int(self.value, base=2) * 2 ** (- (self.n - self.p))


port = PortWordLength(n=5, p=2, value="11011")
print(f"The word length (n = {port.n}, p = {port.p}) of the bits \"{port.value}\" is: ")
print(port.value_num())

The word length (n = 5, p = 2) of the bits "11011" is: 
3.375


## Forming contract for capturing error of fixed-point number

Here we introduce how to form the contract for the fixed-point number operation.
First, we need to model how the error is generated.
Then we consider the propagation of the error.
We formulate the contract by combining the generation of error and propagation of the error.

#### Error Modeling
Each operation can be seen as a two-staged process.
First, an ideal output with certian wordlength is generated without losing any information, given the word lengths of all input ports.
Then the ideal output is truncated to the actual output port and could result in truncation error.
Therefore, the modeling of the ideal output wordlength and the truncation error is required to formulate the contract.

#### Truncation Error Modeling
In the following, we details how to model the truncation error.

##### Case 1. Binary Point Position Unchanged
In this case the change of word length only result in losing the lowest significant bits.
The maximal truncation error from the word length $(n_1, p)$ to the word length $(n_2, p)$ is $2^{p}(2^{-n_2}-2^{-n_1})$, when $n_1 > n_2$.
If $n_1 <= n_2$, there is no truncation and therefore the error is exactly $0$.

The following example shows how the error is bounded:

In [24]:
def truncation_error_same_position(pi, po):
    assert(pi.p == po.p)
    if pi.n > po.n:
        return 2 ** po.p * (2 ** (-po.n) - 2 ** (-pi.n))
    else :
        return 0

def truncate(pi, po):
    # separate the bits into two parts, before binary point and after binary point
    bits_before_point = pi.value[:pi.p]
    bits_after_point = pi.value[pi.p:]
    
    # truncate or appending 0 
    if po.p >= pi.p:
        bits_before_point = ("0" * (po.p - pi.p)) + bits_before_point
    else:
        bits_before_point = bits_before_point[pi.p - po.p:]

    if po.n - po.p >= pi.n - pi.p:
        bits_after_point = bits_after_point + "0" * (po.n - po.p - pi.n + pi.p)
    else:
        bits_after_point = bits_after_point[:(po.n - po.p)]

    # combine the both parts
    ret = bits_before_point + bits_after_point
    po.set_value(value = ret)
    return po.value

In [29]:
p1 = PortWordLength(n=7, p=2, name="p1", value = "1111111")
p2 = PortWordLength(n=5, p=2, name="p2")

print("Truncation Error: ")
print(truncation_error_same_position(p1, p2))
print("Example Value to produce the error")

truncate(p1, p2)
print(f"p1 = {p1.value_num()}, bits: {p1.value}")
print(f"p2 = {p2.value_num()}, bits: {p2.value}")
print(f"p1 - p2 = ", p1.value_num() - p2.value_num())

Truncation Error: 
0.09375
Example Value to produce the error
p1 = 3.96875, bits: 1111111
p2 = 3.875, bits: 11111
p1 - p2 =  0.09375


Then let's see what happen if the binary point position is changed
##### Case 2. Binary Point Position Change
Consider the case when $(n_1, p_1)$ to the word length $(n_2, p_2)$
If the binary point position is change and result in less bits before the binary point ($p_1 > p_2$), the significant bits in the initial representation will be lost. 
Therefore, it is reasonable to assume that we know exactly the maximum number of the initial number should not set bits that cannot be included in $2^{p_2}$, which sets an assumption that the initial number must be strictly less than $2^{p_2}$.
Then, we can see the fixed-point number as if the those uneccesary most significant bits ($p_1 - p_2$) can be discarded.
$(n_1, p_1)$ can be seen as $(n_1 - p_1 + p_2, p_2)$. 
Then it become the case one, only with the additional assumption.

The following example shows how the error can be calculated.

In [56]:
def truncation_error(pi, po):
    # remove uneccesary most significant bits
    # Remider: The assumption must hold!!
    pi_adjusted = PortWordLength(n=pi.n - pi.p + po.p, p=po.p)
    # return the truncation error as in case 1
    return truncation_error_same_position(pi=pi_adjusted, po=po)

def get_assumption_value(pi, po):
    if pi.p > po.p:
        return 2 ** po.p
    else:
        return float("inf") # no additional assumption needed

def check_assumption_value(pi, po):
    assumption_value = get_assumption_value(pi, po)
    print(f"Assumption: {pi.name} < {assumption_value}")
    if not pi.value_num() < assumption_value:
        print("Assumption failed, truncation of MSB occurs")
        return False
    else:
        print(f"Assumption Satisfied ({pi.name} = {pi.value_num()} < {assumption_value})")
        return True
        


In [59]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")

assert(check_assumption_value(pi=p1, po=p2))

print("Truncation Error: ")
print(truncation_error(p1, p2))
print("Example Value to produce the error")

truncate(p1, p2)
print(f"p1 = {p1.value_num()}, bits: {p1.value}")
print(f"p2 = {p2.value_num()}, bits: {p2.value}")
print(f"p1 - p2 = ", p1.value_num() - p2.value_num())

Assumption: p1 < 4
Assumption Satisfied (p1 = 2.4375 < 4)
Truncation Error: 
0.0625
Example Value to produce the error
p1 = 2.4375, bits: 0100111
p2 = 2.375, bits: 10011
p1 - p2 =  0.0625


#### Ideal Output Wordlength
After we analyze the truncation error, the next problem is what is the wordlength before truncation?
Each operation requires an minimal ideal output wordlength to fully hold the resulting value without losing any information.
We represent the ideal output wordlength using $(n^*, p^*)$.
This is the wordlength truncated before sending to the output port.
And $(n_1, p_1), (n_2, n_2)$ denotes the wordlength of two inputs.

The ideal output wordlength depends on the type of operation
Let's first examine the addition and then the multiplication

##### Addition
The following equations show the ideal output wordlnegth for addition.

$n^* = \max(n_1, n_2 - p_2 + p_1) + \min(0, p_1 - p_2) + 1$,

$p* = \max(p_1, p_2) + 1$.

The rationale behind the equations is to keep every bits in both inputs and add an additional one bits in the front of the output to handle the carry.

Using the ideal output wordlength and the above truncation error. We can calculate the error induced by addition

In [72]:
def compute_required_word_length_add(in1: PortWordLength, in2:PortWordLength) -> PortWordLength:
    new_n = max(in1.n, in2.n - in2.p + in1.p) - min(0, in1.p - in2.p) + 1
    new_p = max(in1.p, in2.p) + 1
    return PortWordLength(n=new_n, p=new_p)

def error_add(p1, p2, po):
    print(f"Input {p1.name}: (n={p1.n}, p={p1.p})")
    print(f"Input {p2.name}: (n={p2.n}, p={p2.p})")
    p_ideal = compute_required_word_length_add(in1=p1, in2=p2)
    print(f"Ideal Output: (n={p_ideal.n}, p={p_ideal.p})")
    print(f"Actual Output ({po.name}): (n={po.n}, p={po.p})")

    assumption_value = get_assumption_value(pi=p_ideal, po=po)
    print(f"Assumption: {p1.name} + {p2.name} < {assumption_value}")
    return truncation_error(pi=p_ideal, po=po)
    

In [75]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

err = error_add(p1=p1, p2=p2, po=p3)

print(f"Error of the addition calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=8, p=4)
Actual Output (p3): (n=5, p=3)
Assumption: p1 + p2 < 8
Error of the addition calculation: 0.1875



##### Multiplication
The following equations show the ideal output wordlnegth for multiplication.
$n^* = n_1 + n_2$,
$p* = p_1, p_2$.

In [78]:
def compute_required_word_length_mult(in1: PortWordLength, in2:PortWordLength) -> PortWordLength:
    new_n = in1.n + in2.n
    new_p = in1.p + in2.p
    return PortWordLength(n=new_n, p=new_p)

def error_mult(p1, p2, po):
    print(f"Input {p1.name}: (n={p1.n}, p={p1.p})")
    print(f"Input {p2.name}: (n={p2.n}, p={p2.p})")
    p_ideal = compute_required_word_length_mult(in1=p1, in2=p2)
    print(f"Ideal Output: (n={p_ideal.n}, p={p_ideal.p})")
    print(f"Actual Output ({po.name}): (n={po.n}, p={po.p})")

    assumption_value = get_assumption_value(pi=p_ideal, po=po)
    print(f"Assumption: {p1.name} * {p2.name} < {assumption_value}")
    return truncation_error(pi=p_ideal, po=po)

In [79]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

err = error_mult(p1=p1, p2=p2, po=p3)

print(f"Error of the multiplication calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=12, p=5)
Actual Output (p3): (n=5, p=3)
Assumption: p1 * p2 < 8
Error of the multiplication calculation: 0.2421875


#### Propagation of Error

The error $e$ already presents in a signal to the port $p$ will propagate to the final output.
Such error including error from the previus operation, quantization error and known inaccuracies of the input.
This error is independent to the error of truncation error, as truncation error compares the value of the output to the nominal value of the input signal.
As a result, the error can be represented as the sum of the propagation error and the truncation error in the stage: 

$e = e_p + e_t$

, where $e_p$ is the propagtion error from inputs regardless of how many bits we have in the output, and $e_t$ denotes the truncation error that occurs due to the descrepancy between the wordlength of the ideal output and that of the actual output.
Thus, it is important to explore differnet word length in the circuit, as one change to the word length affect the accuracy of the output.

Let's now see how the error is propagated in different operation

##### Propagation of Error in Addition
The error is exactly the sum of error from both inputs.
$e_p = e_1 + e_2$


In [ ]:
p1 = PortWordLength(n=7, p=3, e=0.5, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, e=0.2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

err = error_add(p1=p1, p2=p2, po=p3)

print(f"Error of the addition calculation: {err}")

##### General operation
Using the above analysis, we now formulate the contract for different operation.

In the following, we import the tools for calculating the value for each operation given the word length of the input and output.

We use the class PortWordLength to represent the word length.
It also contains the known error of the number for propagating the error.

#### Example 1
Consider the following system with only two adder.
TODO: put figure

The following function is used to generate contracts, considering all error types.

In [ ]:
def form_contract(in_port1, in_port2, out_port, operation):
    ret_contract = {}
    ret_contract["InputVars"] = [f"{in_port1.name}_a", f"{in_port1.name}_e",
                                 f"{in_port2.name}_a", f"{in_port2.name}_e"]
    ret_contract["OutputVars"] = [f"{out_port.name}_a", f"{out_port.name}_e"]
    a_bound = get_assumption_bound(in_port_1=in_port1, in_port_2=in_port2, out_port=out_port, operation_length_fn=operation)
    ret_contract["assumptions"] =  [{"coefficients":{f"{in_port1.name}_a":1, f"{in_port2.name}_a":1},
                                   "constant":a_bound}]
    if operation == "add":
        error_bound = get_guarantee_bound(in_port_1=in_port1, in_port_2=in_port2, out_port=out_port, operation_length_fn=operation)
        actual_val_bound = get_actual_possible_value(in_port=out_port)
        ret_contract["guarantees"] =  [{"coefficients":{f"{in_port1.name}_e":-1, f"{in_port2.name}_e":-1, f"{out_port.name}_e": 1},
                                    "constant":error_bound},
                                    {"coefficients":{f"{out_port.name}_a": 1}, "constant":actual_val_bound},
                                    ]
    elif operation == "mult":
        pass

    return ret_contract

In [ ]:
    contracts = []
    p1 = PortWordLength(n=5, p=2, name = "p1")
    p2 = PortWordLength(n=5, p=3, name = "p2")
    p3 = PortWordLength(n=5, p=3, name = "p3")
    c1 = form_contract(in_port1=p1, in_port2=p2, out_port=p3, operation="add")
    contracts.append(c1)
    p4 = PortWordLength(n=7, p=3, name = "p4")
    p5 = PortWordLength(n=6, p=3, name = "p5")
    c2 = form_contract(in_port1=p3, in_port2=p4, out_port=p5, operation="add")

In [ ]:
contract1 = readContract(c1)
contract2 = readContract(c2)


In [ ]:
print("Contract 1:\n" + str(contract1))
print("Contract 2:\n" + str(contract2))

In [ ]:
contract_sys = contract1.compose(contract2)
print("Contract Sys:\n" + str(contract_sys))